# EOEPCA Data Access Validation and Usage Notebook

## Setup

In [4]:
import os
import requests
import json
from pathlib import Path

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [5]:
load_eoepca_state()

In [6]:
platform_domain = os.environ.get("INGRESS_HOST")
data_access_domain = f'{os.environ.get("HTTP_SCHEME")}://eoapi.{platform_domain}'

print(f"Data Access API: {data_access_domain}")

Data Access API: https://eoapi.notebook-test.develop.eoepca.org


## Validate Data Access API Endpoints

In [7]:
endpoints = [
    ("STAC API Landing Page", f"{data_access_domain}/stac/"),
    ("STAC Swagger UI", f"{data_access_domain}/stac/api.html"),
    ("Raster API Swagger UI", f"{data_access_domain}/raster/api.html"),
    ("Vector API Swagger UI", f"{data_access_domain}/vector/api.html")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

STAC API Landing Page (https://eoapi.notebook-test.develop.eoepca.org/stac/): 200
STAC Swagger UI (https://eoapi.notebook-test.develop.eoepca.org/stac/api.html): 200
Raster API Swagger UI (https://eoapi.notebook-test.develop.eoepca.org/raster/api.html): 200
Vector API Swagger UI (https://eoapi.notebook-test.develop.eoepca.org/vector/api.html): 200


## STAC Items Search (Before Ingestion)

In [8]:
search_url = f"{data_access_domain}/stac/search"
search_payload = {
    "bbox": [-25, 63, -24, 65],
    "datetime": "2023-01-01T00:00:00Z/2024-01-01T00:00:00Z",
    "limit": 5
}

response = requests.post(search_url, json=search_payload)
print(f"STAC search status code (before ingestion): {response.status_code}")
print(json.dumps(response.json(), indent=2))

STAC search status code (before ingestion): 200
{
  "type": "FeatureCollection",
  "links": [
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/"
    },
    {
      "rel": "self",
      "type": "application/json",
      "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/search"
    }
  ],
  "features": [],
  "numberMatched": 0,
  "numberReturned": 0
}


## Load and Inspect Sample STAC Collection and Items

In [9]:
collections_url = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/scripts/data-access/collections/sentinel-2-iceland/collections.json"
items_url = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/scripts/data-access/collections/sentinel-2-iceland/items.json"

collections_json = requests.get(collections_url).json()
items_json = requests.get(items_url).json()

print(json.dumps(collections_json, indent=2)[:500])
print(json.dumps(items_json, indent=2)[:500])

{
  "id": "sentinel-2-iceland",
  "type": "Collection",
  "links": [
    {
      "rel": "items",
      "type": "application/geo+json",
      "href": "items.json"
    },
    {
      "rel": "cite-as",
      "href": "https://doi.org/10.5270/S2_-742ikth",
      "title": "Copernicus Sentinel-2 MSI Level-2A (L2A) Bottom-of-Atmosphere Radiance"
    },
    {
      "rel": "license",
      "href": "https://sentinel.esa.int/documents/247904/690755/Sentinel_Data_Legal_Notice",
      "title": "proprietary"
 
[
  {
    "id": "S2A_26WPS_20231109_0_L2A",
    "bbox": [
      -24.944141354917324,
      64.11035031942181,
      -24.177092180629256,
      64.8188325356334
    ],
    "type": "Feature",
    "links": [
      {
        "rel": "collection",
        "type": "application/json",
        "href": "collections.json"
      }
    ],
    "assets": {
      "aot": {
        "href": "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/26/W/PS/2023/11/S2A_26WPS_20231109_0_L2A/AOT.tif",
  

## Automated Ingestion via STAC API

In [11]:
collections_endpoint = f"{data_access_domain}/stac/collections"
ingested_collection = requests.post(collections_endpoint, json=collections_json)
print(f"Collection ingestion status code: {ingested_collection.status_code}")
print(ingested_collection.json())

items_endpoint = f"{data_access_domain}/stac/collections/{collections_json['id']}/items"

successful_items = []
failed_items = []

# only take first 5 items for testing
items_json = items_json[:5]

for item in items_json:
    response = requests.post(items_endpoint, json=item)
    if response.status_code == 200:
        successful_items.append(item['id'])
    else:
        failed_items.append((item['id'], response.text))

print(f"Successfully ingested items: {successful_items}")
if failed_items:
    print(f"Failed items: {failed_items}")

Collection ingestion status code: 409
{'code': 'ConflictError', 'description': ''}
Successfully ingested items: []
Failed items: [('S2A_26WPS_20231109_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_27VWL_20231109_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_26WPS_20231108_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_26WNT_20231108_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_26WPT_20231108_0_L2A', '{"code":"ConflictError","description":""}')]


## Validate STAC Collection and Items (Post-Ingestion)

In [12]:
collection_check = requests.get(f"{collections_endpoint}/{collections_json['id']}")
print(f"Collection retrieval status: {collection_check.status_code}")
print(json.dumps(collection_check.json(), indent=2))

items_check = requests.get(items_endpoint)
print(f"Items retrieval status: {items_check.status_code}")
print(json.dumps(items_check.json(), indent=2))

Collection retrieval status: 200
{
  "id": "sentinel-2-iceland",
  "type": "Collection",
  "links": [
    {
      "rel": "items",
      "type": "application/geo+json",
      "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/collections/sentinel-2-iceland/items"
    },
    {
      "rel": "parent",
      "type": "application/json",
      "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/"
    },
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/"
    },
    {
      "rel": "self",
      "type": "application/json",
      "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/collections/sentinel-2-iceland"
    },
    {
      "rel": "items",
      "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/items.json",
      "type": "application/geo+json"
    },
    {
      "rel": "cite-as",
      "href": "https://doi.org/10.5270/S2_-742ikth",
      "title": "Copernicus Sentinel-2 MSI 

## STAC Items Search (After Ingestion)

In [13]:
response = requests.post(search_url, json=search_payload)
print(f"STAC search status code (after ingestion): {response.status_code}")
search_results = response.json()
print(json.dumps(search_results, indent=2))

STAC search status code (after ingestion): 200
{
  "type": "FeatureCollection",
  "links": [
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/"
    },
    {
      "rel": "self",
      "type": "application/json",
      "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/search"
    }
  ],
  "features": [
    {
      "id": "S2A_26WPS_20231109_0_L2A",
      "bbox": [
        -24.944141354917324,
        64.11035031942181,
        -24.177092180629256,
        64.8188325356334
      ],
      "type": "Feature",
      "links": [
        {
          "rel": "collection",
          "type": "application/json",
          "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/collections/sentinel-2-iceland"
        },
        {
          "rel": "parent",
          "type": "application/json",
          "href": "https://eoapi.notebook-test.develop.eoepca.org/stac/collections/sentinel-2-iceland"
        },
  

## Raster and Vector API Availability Checks

In [14]:
raster_health_url = f"{data_access_domain}/raster/healthz"
vector_health_url = f"{data_access_domain}/vector/healthz"

raster_health = requests.get(raster_health_url)
vector_health = requests.get(vector_health_url)

print(f"Raster API Health Check: {raster_health.status_code} - {raster_health.text}")
print(f"Vector API Health Check: {vector_health.status_code} - {vector_health.text}")


Raster API Health Check: 200 - {"database_online":true}
Vector API Health Check: 200 - {"ping":"pong!"}


## STAC Manager UI Check

In [15]:
stac_manager_ui_url = data_access_domain
response = requests.get(stac_manager_ui_url)
print(f"STAC Manager UI status code: {response.status_code}")
response.url

STAC Manager UI status code: 200


'https://eoapi.notebook-test.develop.eoepca.org/'